In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
ratings_movies = pd.read_csv(r'c:\Users\Administrator.DESKTOP-8R20EE2\Documents\sf_data_science\PY_12_Python_Advanced_for_Data_Analysis\ratings_movies\ratings_movies.csv')

In [5]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [7]:
ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies.info()
ratings_movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


,Unnamed: 0,userId,movieId,rating,date,title,genres,year_release
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance,1995.0
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller,1995.0
3,3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,1995.0
4,4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,1995.0


In [11]:
movies_filtered = ratings_movies[ratings_movies['year_release'] == 1999]
grouped = movies_filtered.groupby('title')['rating'].mean().reset_index()
sorted_grouped = grouped.sort_values('rating', ascending=True)
worst_movie = sorted_grouped.iloc[0]['title']
display(sorted_grouped)

,title,rating
41,Bloodsport: The Dark Kumite (1999),0.5
204,Simon Sez (1999),1.0
54,Chill Factor (1999),1.0
213,"Source, The (1999)",1.0
242,Trippin' (1999),1.0
...,...,...
241,Trailer Park Boys (1999),5.0
137,Larry David: Curb Your Enthusiasm (1999),5.0
224,Sun Alley (Sonnenallee) (1999),5.0
101,George Carlin: You Are All Diseased (1999),5.0


In [12]:
movies_filtered_2 = ratings_movies[ratings_movies['year_release'] == 2010]
grouped_2 = movies_filtered_2.groupby('genres')['rating'].mean().reset_index()
sorted_grouped_2 = grouped_2.sort_values('rating', ascending=True)
display(sorted_grouped_2)

,genres,rating
46,Action|Sci-Fi,1.000000
14,Action|Adventure|Horror,1.500000
36,Action|Drama|Fantasy,1.500000
87,Crime|Romance,1.500000
55,Adventure|Comedy|Fantasy,1.833333
...,...,...
82,Crime,4.750000
78,Comedy|Musical,5.000000
67,Animation|Drama|Fantasy|Mystery,5.000000
52,Adventure|Children|Comedy|Mystery,5.000000


In [13]:
user_grouped = ratings_movies.groupby('userId')['genres'].nunique().reset_index()
sorted_grouped_user = user_grouped.sort_values('genres', ascending=False)
display(sorted_grouped_user)

,userId,genres
598,599,524
413,414,482
447,448,403
379,380,399
473,474,395
...,...,...
577,578,15
11,12,15
84,85,13
213,214,13


In [14]:
low_rating_user = ratings_movies.groupby('userId')['rating'].agg(['count','mean']).reset_index()
sorted_low_rating_user = low_rating_user.sort_values(by=['count', 'mean'], ascending=[True, False])

display(sorted_low_rating_user)

,userId,count,mean
52,53,20,5.000000
594,595,20,4.200000
188,189,20,4.100000
568,569,20,4.000000
277,278,20,3.875000
...,...,...,...
273,274,1346,3.235884
447,448,1864,2.847371
473,474,2108,3.398956
598,599,2478,2.642050


In [16]:
filtered_movies = ratings_movies[ratings_movies['year_release'] == 2018]
grouped_filtered_movies = filtered_movies.groupby('genres')['rating'].agg(['count', 'mean']).reset_index()
count_rating = grouped_filtered_movies[grouped_filtered_movies['count'] > 10]
sorted_count_rating = count_rating.sort_values('mean', ascending=False)
display(sorted_count_rating)

,genres,count,mean
5,Action|Adventure|Sci-Fi,14,3.928571
10,Action|Comedy|Sci-Fi,12,3.875000


In [17]:
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])
ratings_movies['year_rating'] = ratings_movies['date'].dt.year
ratings_movies_pivot = ratings_movies.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres',
    aggfunc=['mean']
)

display(ratings_movies_pivot)

mean                             \
genres      (no genres listed)    Action Action|Adventure   
year_rating                                                 
1996                       NaN  2.730769         3.454545   
1997                       NaN  3.538462         4.150000   
1998                       NaN       NaN         4.200000   
1999                       NaN       NaN         4.000000   
2000                       NaN  2.588235         3.738462   
2001                       NaN  3.000000         3.500000   
2002                       NaN  2.750000         4.304348   
2003                       NaN  3.833333         3.277778   
2004                       NaN  2.700000         4.136364   
2005                       NaN  3.357143         3.413043   
2006                       NaN  2.166667         4.090909   
2007                       NaN  2.583333         3.666667   
2008                       NaN  3.000000         3.656250   
2009                       NaN       NaN         3.714286   
2010                       NaN  3.500000         3.437500   
2011                       NaN  4.500000         3.888889   
2012                       NaN  2.625000         3.900000   
2013                       NaN  2.500000         3.833333   
2014                       NaN       NaN         3.875000   
2015                  0.500000  3.100000         3.560976   
2016                  3.500000  3.545455         3.722222   
2017                  3.000000  2.637931         3.527778   
2018                  3.676471  2.588235         3.397436   

                                                                            \
genres      Action|Adventure|Animation Action|Adventure|Animation|Children   
year_rating                                                                  
1996                               NaN                                 NaN   
1997                               NaN                                 NaN   
1998                               NaN                                 NaN   
1999                               NaN                                 NaN   
2000                               NaN                                 NaN   
2001                               NaN                                 NaN   
2002                               NaN                                 NaN   
2003                               NaN                                 NaN   
2004                               NaN                                 NaN   
2005                               NaN                                 NaN   
2006                               NaN                                 NaN   
2007                               NaN                                 NaN   
2008                               NaN                                 NaN   
2009                               NaN                                 NaN   
2010                               NaN                                 NaN   
2011                               NaN                                 NaN   
2012                               NaN                            4.000000   
2013                               NaN                            3.000000   
2014                             3.000                            3.500000   
2015                             3.800                            3.666667   
2016                             3.750                            3.833333   
2017                             3.375                            3.500000   
2018                             3.650                            3.166667   

                                                        \
genres      Action|Adventure|Animation|Children|Comedy   
year_rating                                              
1996                                               NaN   
1997                                               NaN   
1998                                               NaN   
1999                                               NaN   
2000